
## **1. Data Collection**
### **1.1. Get the list of Michelin restaurants**

In [5]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from IPython.display import display

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
} # user agent is used to simulate that the http request comes from a real web browser, this prevent the server from blocking requests

def guide_michelin(): # 2037
        links = []
        for i in range(1,101): #100
            link = "https://guide.michelin.com/en/it/restaurants/page/{}".format(i)
            try:
                response = requests.get(link, headers=headers)
            except Exception as e:
                print(f"{e} \n {link}")
                continue
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                section = soup.find('div', class_="row restaurant__list-row js-restaurant__list_items")
                if section:  
                    for a_tag in section.find_all('a', href=True):
                        href = 'https://guide.michelin.com' + a_tag['href']
                        if href not in links and "/restaurant/" in href: 
                            links.append(href)
            else:
                print(f"Failed to retrieve page {i}")    
        return links

url_set = guide_michelin()
print(len(url_set))

1981


In [7]:
with open('links.txt', 'w') as f:
    for url in url_set:
        f.write(url + '\n')

### **1.2. Crawl Michelin restaurant pages**

In [8]:
if not os.path.exists('pages'):
    os.makedirs('pages')

with open('links.txt', 'r') as f:
    urls = f.read().splitlines()

# Create directories and save HTML documents
for index, url in enumerate(urls):
    page_number = index // 20 + 1
    directory = os.path.join('pages', f'page_{page_number}')
    if not os.path.exists(directory):
        os.makedirs(directory)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            file_path = os.path.join(directory, f'document_{index}.html')
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)
        else:
            print(f"Failed to retrieve {url}")
    except Exception as e:
        print(f"Error fetching {url}: {e}")

print("HTML documents saved successfully.")

HTML documents saved successfully.


In [9]:
dir_paths = [os.path.join('pages', dir) for dir in os.listdir('pages')]
len(dir_paths)

100

### **1.3. Parse downloaded pages**

In [10]:
# Function to extract restaurant details from HTML content
def extract_restaurant_details(content):
    
    # Extract the restaurant name
    name = content.find('h1', class_='data-sheet__title').get_text(strip=True) if content.find('h1', class_='data-sheet__title') else ""
    
    # Extract the first row of basic information
    firstRow = content.find_all("div", class_="data-sheet__block--text")[0].get_text(strip=True)
    #firstRow = content.find("div", class_="data-sheet__block--text").get_text(strip=True)
    firstRow_list = [info.strip() for info in firstRow.split(",")]

    address = " ".join(firstRow_list[:-3]) if len(firstRow_list) > 3 else ""
    city = firstRow_list[-3] if len(firstRow_list) > 2 else ""
    postalCode = firstRow_list[-2] if len(firstRow_list) > 1 else ""
    country = firstRow_list[-1] if firstRow_list else ""

    # Extract the second row of basic information
    secondRow = content.find_all("div", class_="data-sheet__block--text")[1].get_text(strip=True)
    #secondRow = content.find("div", class_="data-sheet__block--text").get_text(strip=True)
    secondRow_list = [info.strip() for info in secondRow.split("·")]

    priceRange = secondRow_list[0] if secondRow_list else ""
    cuisineType = secondRow_list[1] if len(secondRow_list) > 1 else ""

    # Extract the description
    description = content.find("div", class_="data-sheet__description").get_text(strip=True) if content.find("div", class_="data-sheet__description") else ""

    # Extract facilities and services
    facilitiesServices_div = content.find_all("div", class_="col col-12 col-lg-6")
    # facilitiesServices_div = content.find("div", class_="col col-12 col-lg-6")
    facilitiesServices = [li.get_text(strip=True) for li in facilitiesServices_div[0].find_all("li")] if facilitiesServices_div else []
    # facilitiesServices = [li.get_text(strip=True) for li in facilitiesServices_div.find("li")] if facilitiesServices_div else []

    # Extract credit card information
    creditCards_div = content.find("div", class_="restaurant-details__services--info")
    creditCards = [os.path.basename(img["data-src"]).split("-")[0] for img in creditCards_div.find_all("img")] if creditCards_div else []

    # Extract phone number
    phoneNumber = content.find("span", attrs={"x-ms-format-detection": "none"}).get_text(strip=True) if content.find("span", attrs={"x-ms-format-detection": "none"}) else ""

    # Extract website
    website_div = content.find("div", class_="collapse__block-item link-item")
    website = website_div.find("a", class_="link js-dtm-link")["href"] if website_div and website_div.find("a", class_="link js-dtm-link") else ""

    # Return the extracted data as a dictionary
    return {
        "restaurantName": name,
        "address": address,
        "city": city,
        "postalCode": postalCode,
        "country": country,
        "priceRange": priceRange,
        "cuisineType": cuisineType,
        "description": description,
        "facilitiesServices": facilitiesServices,
        "creditCards": creditCards,
        "phoneNumber": phoneNumber,
        "website": website
    }

# Collecting data from all HTML files
#folder_paths = [d for d in os.listdir('pages') if os.path.isdir(d) and d.startswith("page_")]
dir_paths = [os.path.join('pages', dir) for dir in os.listdir('pages')]

data = []
for dir in dir_paths:
    for html_file in os.listdir(dir):
        if html_file.endswith(".html"):
            with open(os.path.join(dir, html_file), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                restaurant_details = extract_restaurant_details(soup)
                data.append(restaurant_details)

# Create a DataFrame from the data list
df = pd.DataFrame(data)

df.columns = ["restaurantName", "address", "city", "postalCode", "country", "priceRange", "cuisineType", "description", "facilitiesServices", "creditCards", "phoneNumber", "website"]

In [11]:
# Display the DataFrame
display(df)

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Hydra,via Antonio Mazza 30,Salerno,84121,Italy,€€,"Campanian, Contemporary",Situated in the picturesque historic centre of...,"[Air conditioning, Restaurant offering vegetar...","[amex, dinersclub, mastercard, visa]",+39 089 995 8437,http://www.ristorantehydra.com
1,Gimmi Restaurant,via San Pietro in Lama 23,Lecce,73100,Italy,€€€,Contemporary,Despite its location in a Dominican monastery ...,"[Air conditioning, Terrace, Wheelchair access]","[amex, maestrocard, mastercard, visa]",+39 0832 700920,https://www.chiostrodeidomenicani.it/ristorante/
2,Felix Lo Basso home & restaurant,via Carlo Goldoni 36,Milan,20129,Italy,€€€€,"Italian Contemporary, Creative",Brilliant chef Felix Lo Basso’s menu is inspir...,"[Air conditioning, Counter dining, Wheelchair ...","[amex, mastercard, visa]",+39 02 4540 9759,https://www.felixlobassorestaurant.it/
3,L'Acciuga,via Settevalli 217,Perugia,06128,Italy,€€€,"Contemporary, International",You would never guess that there was a gourmet...,"[Air conditioning, Interesting wine list, Terr...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 339 263 2591,https://www.lacciuga.net/
4,Antiche Sere,via Cenischia 9,Turin,10139,Italy,€,"Piedmontese, Classic Cuisine",This renowned osteria situated in a district o...,"[Air conditioning, Terrace]","[dinersclub, mastercard, visa]",+39 011 385 4347,
...,...,...,...,...,...,...,...,...,...,...,...,...
1978,Vintage 1997,piazza Solferino 16/h,Turin,10121,Italy,€€€,"Italian, Classic Cuisine",The several tasting menus at this restaurant i...,"[Air conditioning, Interesting wine list, Rest...","[amex, mastercard, visa]",+39 011 535948,https://www.vintage1997.com/
1979,Locanda Margon,via Margone 15,Ravina,38123,Italy,€€€€,"Creative, Contemporary",This restaurant with views of Trento and the A...,"[Air conditioning, Car park, Garden or park, G...","[amex, dinersclub, mastercard, visa]",+39 0461 349401,https://www.locandamargon.it/
1980,Bon Wei,via Castelvetro 16/18,Milan,20154,Italy,€€,"Chinese, Asian",China on a plate! This attractive restaurant w...,"[Air conditioning, Wheelchair access]","[amex, mastercard, visa]",+39 02 341308,https://www.bon-wei.it/
1981,Le Lampare al Fortino,via Tiepolo molo Sant'Antonio,Trani,76125,Italy,€€€,"Mediterranean Cuisine, Modern Cuisine","Built over a medieval church, this old fort th...","[Air conditioning, Great view, Interesting win...","[amex, dinersclub, mastercard, visa]",+39 0883 480308,https://www.lelamparealfortino.it/it/home/


In [4]:
%pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 2  Search Engine

### 2.0 Preprocessing the Text

In [12]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [13]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and remove stopwords, then apply stemming
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

# Apply to the description field
df['processed_description'] = df['description'].apply(preprocess_text)


### 2.1 Conjunctive Query

### 2.1.1 Create the Index!

In [14]:
from collections import defaultdict
import pandas as pd

vocabulary = {}
inverted_index = defaultdict(list)
term_id_counter = 0

for doc_id, description in enumerate(df['processed_description']):
    for word in description.split():
        # Map each unique word to a term_id
        if word not in vocabulary:
            vocabulary[word] = term_id_counter
            term_id_counter += 1
        term_id = vocabulary[word]
        inverted_index[term_id].append(doc_id)

# Save the vocabulary to a CSV file
pd.DataFrame(list(vocabulary.items()), columns=['term', 'term_id']).to_csv('vocabulary.csv', index=False)


In [15]:
import json

with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)


### 2.1.2 Execute the Query

In [16]:
def preprocess_query(query):
    query = query.lower()
    query = query.translate(str.maketrans('', '', string.punctuation))
    tokens = [stemmer.stem(word) for word in query.split() if word not in stop_words]
    return tokens

def conjunctive_query(query):
    query_terms = preprocess_query(query)
    term_ids = [vocabulary.get(term) for term in query_terms if term in vocabulary]

    if not term_ids:
        return pd.DataFrame(columns=["restaurantName", "address", "description", "website"])

    # Start with the document list for the first term, then intersect with others
    matching_docs = set(inverted_index[term_ids[0]])
    for term_id in term_ids[1:]:
        matching_docs &= set(inverted_index[term_id])

    results = df.loc[list(matching_docs), ["restaurantName", "address", "description", "website"]]
    return results



In [10]:
%pip install scikit-learn



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_description'])


In [18]:
tfidf_index = defaultdict(list)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Loop over each term (feature) in the TF-IDF matrix
for term_id, term in enumerate(feature_names):
    # Get non-zero document indices and the corresponding scores for this term
    doc_indices = tfidf_matrix[:, term_id].nonzero()[0]
    scores = tfidf_matrix[:, term_id].data
    
    # Append each document ID and score to the tfidf_index dictionary 
    for doc_id, score in zip(doc_indices, scores):
        tfidf_index[term].append((doc_id, score))



### 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

def ranked_query(query, top_k=5):
    query_vec = tfidf_vectorizer.transform([preprocess_text(query)])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_doc_indices = cosine_similarities.argsort()[-top_k:][::-1]

    results = df.loc[top_doc_indices, ['restaurantName', 'address', 'description', 'website']]
    results['similarity_score'] = cosine_similarities[top_doc_indices]
    return results



### Testing

In [20]:
# Test the conjunctive query
query = "This pleasant, warmly decorated restaurant is ..."
conjunctive_results = conjunctive_query(query)
print(conjunctive_results)
display(conjunctive_results)
# Test the ranked query
ranked_results = ranked_query(query, top_k=5)
print(ranked_results)
display(ranked_results)


      restaurantName                         address  \
111  Osteria Taviani  piazza Vittorio Emanuele II 28   

                                           description website  
111  This pleasant, warmly decorated restaurant is ...          


,restaurantName,address,description,website
111,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",


       restaurantName                         address  \
111   Osteria Taviani  piazza Vittorio Emanuele II 28   
238          Menzaghi             via San Giovanni 74   
1497       SaleGrosso              viale II Giugno 15   
1673   Marsupino 1901               via Roma Serra 20   
1818   Lio Pellegrini               via San Tomaso 47   

                                            description  \
111   This pleasant, warmly decorated restaurant is ...   
238   Once you’ve found the car park, make your way ...   
1497  One of the town’s authentic favourites, this f...   
1673  The Marsupino family has been running this tra...   
1818  Despite its location in the town centre, the a...   

                                 website  similarity_score  
111                                               0.302989  
238   https://www.ristorantemenzaghi.it/          0.221025  
1497  http://www.ristorantesalegrosso.it          0.203840  
1673  https://www.trattoriamarsupino.it/          0.174293

,restaurantName,address,description,website,similarity_score
111,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",,0.302989
238,Menzaghi,via San Giovanni 74,"Once you’ve found the car park, make your way ...",https://www.ristorantemenzaghi.it/,0.221025
1497,SaleGrosso,viale II Giugno 15,"One of the town’s authentic favourites, this f...",http://www.ristorantesalegrosso.it,0.203840
1673,Marsupino 1901,via Roma Serra 20,The Marsupino family has been running this tra...,https://www.trattoriamarsupino.it/,0.174293
1818,Lio Pellegrini,via San Tomaso 47,"Despite its location in the town centre, the a...",https://www.liopellegrini.it/site/,0.172042


# 3. Define a New Score!


In [21]:
import heapq
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [22]:
def calculate_cosine_similarity(query, tfidf_vectorizer, tfidf_matrix):
    # Convert the query into a TF-IDF vector
    query_tfidf = tfidf_vectorizer.transform([query])
    
     # Compute cosine similarity between the query vector and all document vectors
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    #Return similarity scores for all documents
    return cosine_similarities

In [23]:
# Main function for custom ranking. 
def custom_scoring(query, df, tfidf_vectorizer, tfidf_matrix, top_k=5, cuisine_preferences=None, service_preferences=None, price_preferences=None):
    # Define differents weights for description, cuisine, facilities and price.
    DESCRIPTION_WEIGHT = 0.5
    CUISINE_WEIGHT = 0.2
    FACILITIES_WEIGHT = 0.2
    PRICE_WEIGHT = 0.3

    # Calculate cosine similarities between the query and all descriptions
    cosine_similarities = calculate_cosine_similarity(query, tfidf_vectorizer, tfidf_matrix)
    top_k_restaurants = []

    # Iterate over all documents and calculate the custom score for each
    for doc_id, cosine_score in enumerate(cosine_similarities):
        # Score based on description similarity
        description_score = cosine_score * DESCRIPTION_WEIGHT

        # Score for matching "cuisineType"
        cuisine_score = 0
        if 'cuisineType' in df.columns and cuisine_preferences:
            for pref in cuisine_preferences:
                if pref.lower() in df.loc[doc_id, 'cuisineType'].lower():
                    cuisine_score += CUISINE_WEIGHT

        # Score for matching "facilitiesServices"
        facilities_score = 0
        if 'facilitiesServices' in df.columns and service_preferences:
            facilities = df.loc[doc_id, 'facilitiesServices']
            if isinstance(facilities, list):
                for service in service_preferences:
                    if any(service.lower() == facility.lower() for facility in facilities):
                        facilities_score += FACILITIES_WEIGHT

        # Score for matching "priceRange"
        price_score = 0
        if 'priceRange' in df.columns and price_preferences:
            price_range = df.loc[doc_id, 'priceRange']
            for price in price_preferences:
                if price in price_range:
                    price_score += PRICE_WEIGHT

        # Calculate the final custom score
        final_score = description_score + cuisine_score + facilities_score + price_score

        # Add to the heap to maintain top-k results
        if len(top_k_restaurants) < top_k:
            heapq.heappush(top_k_restaurants, (final_score, doc_id))
        else:
            heapq.heappushpop(top_k_restaurants, (final_score, doc_id))

    # Sort the top-k results by score in descending order
    top_k_restaurants = sorted(top_k_restaurants, key=lambda x: x[0], reverse=True)

    # Prepare the final output
    results = []
    for score, doc_id in top_k_restaurants:
        results.append({
            "restaurantName": df.loc[doc_id, "restaurantName"],
            "address": df.loc[doc_id, "address"],
            "description": df.loc[doc_id, "description"],
            "website": df.loc[doc_id, "website"],
            "custom_score": round(score, 3)
        })

    return pd.DataFrame(results)

In [24]:
#Define the query and the user's preferences
query = "seafood"
cuisine_preferences = ["Modern Cuisine"]
service_preferences = ["Terrace", "Air conditioning"]   
price_preferences = ["€€€"]
top_k = 5

# Call the custom scoring function and display the results
results_df = custom_scoring(query, df, tfidf_vectorizer, tfidf_matrix, top_k, cuisine_preferences, service_preferences,  price_preferences)
display(results_df)

,restaurantName,address,description,website,custom_score
0,L'Osteria di Santa Marina,campo Santa Marina sestiere di Castello 5911,This classy inn is managed by young profession...,https://osteriadisantamarina.com/,1.011
1,Le Lampare al Fortino,via Tiepolo molo Sant'Antonio,"Built over a medieval church, this old fort th...",https://www.lelamparealfortino.it/it/home/,0.981
2,Essenza Bistrot,via delle Terme 8/A,A welcoming and original bistro-restaurant in ...,https://www.essenzabistrot.it,0.976
3,Jamantè,via San Vito 97,Located not far from the historic centre and t...,https://www.jamanteristorante.com,0.967
4,Livello 1,via Duccio di Buoninsegna 25,Situated in the suburbs off the main tourist t...,http://www.ristorantelivello1.it,0.965


## 5 Bonus 


In [25]:
from collections import defaultdict
import os
import pandas as pd
from bs4 import BeautifulSoup

# Function to preprocess text for tokenization and normalization
def preprocess_text(text):
    """Preprocesses the text by converting to lowercase and splitting into tokens."""
    return text.lower().split()

# Function to build vocabularies and inverted indexes
def build_vocabulary_and_inverted_indexes(df):
    """
    Builds vocabularies and inverted indexes for restaurantName, city, and cuisineType fields.
    """
    vocab = {
        'restaurantName': {},
        'city': {},
        'cuisineType': {}
    }
    
    inverted_index = {
        'restaurantName': defaultdict(set),
        'city': defaultdict(set),
        'cuisineType': defaultdict(set)
    }
    
    term_counter = {
        'restaurantName': 0,
        'city': 0,
        'cuisineType': 0
    }

    # Iterate through each row in the DataFrame
    for record_id, row in df.iterrows():
        for field in ['restaurantName', 'city', 'cuisineType']:
            tokens = preprocess_text(row[field]) if pd.notnull(row[field]) else []

            for token in tokens:
                # Add token to vocabulary and assign term ID if not already present
                if token not in vocab[field]:
                    vocab[field][token] = term_counter[field]
                    term_counter[field] += 1

                # Add record ID to the inverted index for this term
                term_id = vocab[field][token]
                inverted_index[field][term_id].add(record_id)

    return vocab, inverted_index

# Function for conjunctive query
def conjunctive_query(query, vocab, inverted_index):
    """
    Performs a conjunctive query to find matching documents based on the query terms.
    """
    query_tokens = preprocess_text(query)
    query_term_ids = [vocab.get(token) for token in query_tokens if token in vocab]

    if not query_term_ids:
        return set()

    doc_sets = [inverted_index[term_id] for term_id in query_term_ids if term_id in inverted_index]

    return set.intersection(*doc_sets) if doc_sets else set()

# Advanced search with filters
def advanced_field_query(data, name_filter='', city_filter='', cuisine_filter=''):
    """
    Filters the DataFrame using the specified filters for restaurantName, city, and cuisineType.
    """
    vocab, inverted_idx = build_vocabulary_and_inverted_indexes(data)
    matching_docs = set(data.index)

    if name_filter:
        name_vocab, name_inv_idx = vocab['restaurantName'], inverted_idx['restaurantName']
        name_results = conjunctive_query(name_filter, name_vocab, name_inv_idx)
        matching_docs.intersection_update(name_results)

    if city_filter:
        city_vocab, city_inv_idx = vocab['city'], inverted_idx['city']
        city_results = conjunctive_query(city_filter, city_vocab, city_inv_idx)
        matching_docs.intersection_update(city_results)

    if cuisine_filter:
        cuisine_vocab, cuisine_inv_idx = vocab['cuisineType'], inverted_idx['cuisineType']
        cuisine_results = conjunctive_query(cuisine_filter, cuisine_vocab, cuisine_inv_idx)
        matching_docs.intersection_update(cuisine_results)

    return data.loc[list(matching_docs), :]

# Function to apply additional filters
def filter_results(data: pd.DataFrame) -> pd.DataFrame:
    """
    Applies additional filters like price range, region, facilities, and credit cards to the results.
    """
    # Collect user input for advanced filters
    restaurant_name_query = input("Enter restaurant name to search: ")
    city_query = input("Enter city to search: ")
    cuisine_type_query = input("Enter cuisine type to search: ")
    
    results = advanced_field_query(data, restaurant_name_query, city_query, cuisine_type_query)
    
    price_range_filter = input("Enter price ranges to filter, separated by spaces (e.g., €, €€, €€€, €€€€): ").split()
    region_filter = input("Enter regions to filter, separated by spaces: ").split()
    region_filter = [region.title() for region in region_filter]
    facilities_filter = input("Enter facilities/services to filter, separated by spaces: ").split()
    facilities_filter = [facility.title() for facility in facilities_filter]
    credit_cards_filter = input("Enter accepted credit cards to filter, separated by spaces: ").split()
    credit_cards_filter = [card.title() for card in credit_cards_filter]

    # Apply filters on the results
    if not results.empty:
        if price_range_filter:
            results = results[results['priceRange'].isin(price_range_filter)]
        if region_filter:
            results = results[results['city'].isin(region_filter)]
        if facilities_filter:
            results = results[results['facilitiesServices'].apply(
                lambda x: all(facility in x for facility in facilities_filter) if isinstance(x, list) else False)]
        if credit_cards_filter:
            results = results[results['creditCards'].apply(
                lambda x: any(card in x for card in credit_cards_filter) if isinstance(x, list) else False)]

    # Display selected columns
    display_columns = results[['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    return display_columns

# Execute the filtering process
filtered_results = filter_results(df)

# Display the filtered results
print(filtered_results)

Empty DataFrame
Columns: [restaurantName, address, cuisineType, priceRange, website]
Index: []


In [26]:
from collections import defaultdict
def create_vocabularies_and_indexes(data):
    vocabularies = {
        'restaurantName': {},
        'city': {},
        'cuisineType': {}
    }
    inverted_indexes = {
        'restaurantName': defaultdict(set),
        'city': defaultdict(set),
        'cuisineType': defaultdict(set)
    }
    term_ids = {
        'restaurantName': 0,
        'city': 0,
        'cuisineType': 0
    }

    # Loop through each restaurant document
    for doc_id, row in data.iterrows():
        for field in ['restaurantName', 'city', 'cuisineType']:
            tokens = preprocess_text(row[field])

            for token in tokens:
                # Assign a new term ID if token not in vocabulary for the field
                if token not in vocabularies[field]:
                    vocabularies[field][token] = term_ids[field]
                    term_ids[field] += 1

                # Add the doc_id to the inverted index for this token in the field
                term_id = vocabularies[field][token]
                inverted_indexes[field][term_id].add(doc_id)

    return vocabularies, inverted_indexes

In [28]:
def conjunctive_query_ad(query, vocabulary, inverted_index):
    # Preprocess the query text and convert it to term IDs using the vocabulary
    query_tokens = preprocess_text(query)
    query_term_ids = [vocabulary.get(token) for token in query_tokens if token in vocabulary]

    # If no terms from the query are found in the vocabulary, return an empty set
    if not query_term_ids:
        return set()

    # Find documents containing all term IDs from the query
    doc_lists = [set(inverted_index[tid]) for tid in query_term_ids if tid in inverted_index]

    # Intersect document lists to get documents that contain all query terms
    if doc_lists:
        matching_docs = set.intersection(*doc_lists)
    else:
        matching_docs = set()

    # Return the set of matching document indices
    return matching_docs

In [29]:
def advanced_field_query(data, restaurant_name_query='', city_query='', cuisine_type_query=''):
    # Create vocabularies and inverted indexes for each field
    vocabularies, inverted_indexes = create_vocabularies_and_indexes(data)
    
    # Initialize results as a set of all document indices
    results = set(data.index)

    # Perform search on `restaurantName` if `restaurant_name_query` is not empty
    if restaurant_name_query:
        vocabR, invIndexR = vocabularies['restaurantName'], inverted_indexes['restaurantName']
        resultsR = conjunctive_query_ad(restaurant_name_query, vocabR, invIndexR)
        results.intersection_update(resultsR)

    # Perform search on `city` if `city_query` is not empty
    if city_query:
        vocabC, invIndexC = vocabularies['city'], inverted_indexes['city']
        resultsC = conjunctive_query_ad(city_query, vocabC, invIndexC)
        results.intersection_update(resultsC)

    # Perform search on `cuisineType` if `cuisine_type_query` is not empty
    if cuisine_type_query:
        vocabCT, invIndexCT = vocabularies['cuisineType'], inverted_indexes['cuisineType']
        resultsCT = conjunctive_query_ad(cuisine_type_query, vocabCT, invIndexCT)
        results.intersection_update(resultsCT)

    # Display the filtered data with the relevant columns
    filtered_data = data.loc[list(results), ['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    return filtered_data

In [30]:
# Prompt user for search criteria
restaurant_name_query = input("Enter restaurant name to search: ")
city_query = input("Enter city to search: ")
cuisine_type_query = input("Enter cuisine type to search: ")

# Run the query
results = advanced_field_query(data, restaurant_name_query, city_query, cuisine_type_query)
results

AttributeError: 'list' object has no attribute 'iterrows'

In [31]:
def filter_results(data : pd.DataFrame )-> pd.DataFrame:
    results = advanced_field_query(data, restaurant_name_query, city_query, cuisine_type_query)
    price_range_filter = input("Enter price ranges to filter, separated by spaces (e.g., €, €€, €€€, €€€€): ").split()
    region_filter = input("Enter regions to filter, separated by spaces: ").split()
    region_filter = [region.title() for region in region_filter]
    facilities_filter = input("Enter facilities/services to filter, separated by spaces: ").split()
    facilities_filter = [facility.title() for facility in facilities_filter]
    credit_cards_filter = input("Enter accepted credit cards to filter, separated by spaces: ").split()
    credit_cards_filter = [card.title() for card in credit_cards_filter]
    # apply the filters on data and display the results 
    filtered_data = data.loc[results.index,:]
    if price_range_filter:
        filtered_data = filtered_data[filtered_data['priceRange'].isin(price_range_filter)]
    if region_filter:
        filtered_data = filtered_data[filtered_data['region'].isin(region_filter)]
    if facilities_filter:
        filtered_data = filtered_data[filtered_data['facilitiesServices'].apply(lambda x: all(facility in x for facility in facilities_filter) if isinstance(x, str) else False)]
        
    if credit_cards_filter:
        filtered_data = filtered_data[filtered_data['creditCards'].apply(lambda x: any(card in x for card in credit_cards_filter) if isinstance(x, str) else False)]

    # Display the filtered results with the relevant columns
    display_columns = filtered_data[['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    return display_columns

In [32]:
filtered_results = filter_results(data)

AttributeError: 'list' object has no attribute 'iterrows'

# Algorithmic Question (AQ)

In [1]:
def collect_packages(num_tests, test_cases):
    outcomes = []

    for case_index in range(num_tests):
        num_packages, package_coords = test_cases[case_index]

        # Sort packages by coordinates (x, y) to ensure the smallest lexicographical path
        package_coords.sort()

        path_steps = []
        reachable = True
        curr_x, curr_y = 0, 0  # Start at (0, 0)

        for target_x, target_y in package_coords:
            # Check if the package is reachable from the current position
            if target_x < curr_x or target_y < curr_y:
                # If any package requires moving left or down, mark as unreachable
                reachable = False
                break

            # Append necessary moves to reach the target package
            path_steps.append('R' * (target_x - curr_x))  # Move right
            path_steps.append('U' * (target_y - curr_y))  # Move up

            # Update the current position to the target package's coordinates
            curr_x, curr_y = target_x, target_y

        if reachable:
            # If all packages are reachable, append "YES" and the path
            outcomes.append("YES\n" + ''.join(path_steps))
        else:
            # If any package is unreachable, append "NO"
            outcomes.append("NO")

    return outcomes

# Sample input data
num_tests = 3
test_cases = [
    (5, [(1, 3), (1, 2), (3, 3), (5, 5), (4, 3)]),
    (2, [(1, 0), (0, 1)]),
    (1, [(4, 3)])
]

# Execute function and print results
results = collect_packages(num_tests, test_cases)
for result in results:
    print(result)


YES
RUUURRRRUU
NO
YES
RRRRUUU


### Pseudocode for the Algorithm

Given a list of packages located on a grid, where each package is represented by coordinates \((x_i, y_i)\), and a robot starting at \((0, 0)\) that can only move right (`R`) and up (`U`):

1. **Input**:
   - \( t \): the number of test cases.
   - For each test case:
      - \( n \): the number of packages.
      - A list of \( n \) packages with coordinates \((x_i, y_i)\).

2. **Algorithm**:
Start

    Function collect_packages(t, test_cases):
        Results = []   // List to store results for each test case
        
        For each test_case in test_cases:
            (n, packages) = test_case     // Extract number of packages and their coordinates
            Sort packages in ascending order by x, then by y    // Lexicographical sorting

            current_x = 0   // Initial robot position (0, 0)
            current_y = 0
            path = ""    // String to build the path
            possible = True

            For each (x, y) in packages:
                If x < current_x or y < current_y:
                    possible = False   // If the package is unreachable (backward movement)
                    Break the loop

                // Add the necessary moves to reach the package (x, y)
                path = path + "R" * (x - current_x)  // Add right movements
                path = path + "U" * (y - current_y)  // Add upward movements

                // Update the robot's position
                current_x = x
                current_y = y

            If possible:
                Append "YES" and path to Results
            Else:
                Append "NO" to Results

        Return Results   // Return results for all test cases

    // Input reading
    t = input()   // Number of test cases
    test_cases = []   // List to store test cases

    For i = 1 to t:
        n = input()   // Number of packages
        packages = []   // List to store package coordinates

        For j = 1 to n:
            x, y = input()   // Package coordinates
            packages.append((x, y))   // Add the package to the list

        Append (n, packages) to test_cases   // Add the test case to the list

    // Call the function and print the results
    results = collect_packages(t, test_cases)
    For each result in results:
        Print result

End


### Proof of Correctness

1. **Sorting Ensures Lexicographical Order**:
   - Sorting the packages by $x$ and then $y$-coordinates ensures that we attempt to collect packages in the lexicographically smallest way.
   - If there is an accessible path for all packages after sorting, this path will be the smallest lexicographical path because it minimizes right and upward movements in order.

2. **Reachability Check**:
   - By moving only right or up from each position, we guarantee that any unreachable package (one that would require moving left or down) will be detected and skipped. The algorithm returns "NO" in such cases.

3. **Path Construction**:
   - For each reachable package, the algorithm appends the correct number of `R` and `U` moves, ensuring that the robot reaches each package in the required order.

4. **Conclusion**:
   - The algorithm is correct because it verifies reachability and constructs the smallest lexicographical path if possible.

### Time Complexity Analysis

1. **Sorting the Packages**:
   - Sorting the list of $n$ packages takes $O (\frac{n}{log(n)})$ time.

2. **Constructing the Path**:
   - The path is constructed by iterating over each package, which takes $O(n)$ time. For each package, we calculate the difference in coordinates and append the required moves.

3. **Overall Complexity**:
   - The sorting step, $O (\frac{n}{log(n)})$, is the most time-consuming operation in the algorithm, making the overall complexity: $O (\frac{n}{log(n)})$

### Verification with a Language Model's Analysis

If you asked a language model to evaluate this code, it would likely arrive at the same time complexity, $O (\frac{n}{log(n)})$ because sorting is the dominant step in this approach. If there were any discrepancies, they would likely stem from misunderstanding the nature of appending moves as an $O(n)$ operation. However, given that sorting $n$ packages is indeed $O (\frac{n}{log(n)})$ and dominates the time complexity, our analysis is accurate.

### Extending the Problem: Robot Can Move Left or Down

With the new rule allowing movement in all directions (right, left, up, down), we consider a **greedy algorithm** where the robot collects the closest package from its current location.

#### Is the Greedy Algorithm Optimal?

1. **Greedy Approach**:
   - At each step, the robot moves to the closest package, defined by the Euclidean distance or Manhattan distance from the current position.
   - The robot continues selecting the nearest uncollected package until all packages are collected.

2. **Counterexample to Greedy Optimality**:
   - The greedy approach does not guarantee the minimum path length for the following reason:
   - **Example**:
     - Suppose there are packages at $(0, 1)$, $(1, 0)$, and $(2, 2)$, and the robot starts at $(0, 0)$.
     - The greedy approach would choose either $(0, 1)$ or $(1, 0)$ first, then the remaining one, and finally $(2, 2)$.
     - However, the optimal path would be to go directly to $(2, 2)$ first, then collect $(0, 1)$ and $(1, 0)$, which would result in a shorter total path.

3. **Conclusion**:
   - The greedy algorithm is **not optimal** for minimizing the total distance. While it may be intuitive and yield a quick solution, it does not guarantee the shortest path because the closest package does not necessarily contribute to the globally shortest route for all packages.

